In [ ]:
# Mean_color = np.zeros((len(x_train),3))
# for i in range(len(x_train)):
#     Mean_color[i,:] = x_train[i].mean(axis=(0,1))

# np.shape(Mean_color)

In [10]:
# fig, axs = plt.subplots(1, 2, figsize=(6, 3))

# axs[0].scatter(Mean_color[:,0], Mean_color[:,1], c=types_train, cmap='viridis')

# sc = axs[1].scatter(Mean_color[:,0], Mean_color[:,1]/Mean_color[:,0], c=types_train, cmap='viridis')

# plt.colorbar(sc)

2

In [ ]:
# _, axs = plt.subplots(len(x_sample), 4, figsize=(15/6*4,5/3*len(x_sample)))

# axs[0,0].set_title('Hue channel')
# axs[0,1].set_title('Saturation channel')
# axs[0,2].set_title('Value channel')
# # axs[0,-2].set_title('Personalized channel')
# axs[0,-1].set_title('Identified image')

# for i in range(len(x_sample)):
#     circles, img_type = localization(x_sample[i])

#     axs[i,-1].imshow(x_sample[i])
#     axs[i,-1].axis('off')

#     for circle in circles:
#         x, y, radius = circle
#         center = (int(x), int(y))
#         area = np.pi*radius**2
#         circle = plt.Circle(center, radius, color='r', fill=False)  # Create a circle object
#         axs[i,-1].add_artist(circle)  # Add the circle to the plot


# plt.tight_layout()
    

In [ ]:
# def classify_image(image_path):
#     # Load image
#     image = cv2.imread(image_path)
#     # Convert to grayscale
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#     # Check uniformity of background (standard deviation of pixel values)
#     std_dev = np.std(gray)
#     if std_dev < 15:
#         return "Plain Background"

#     # Convert image to HSV color space
#     hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     # Define range for skin colors
#     lower_skin = np.array([0, 48, 80], dtype="uint8")
#     upper_skin = np.array([20, 255, 255], dtype="uint8")
    
#     # Create mask for skin colors
#     skinMask = cv2.inRange(hsv, lower_skin, upper_skin)
    
#     # Check the proportion of skin pixels
#     skin_proportion = np.sum(skinMask > 0) / (skinMask.shape[0] * skinMask.shape[1])
#     if skin_proportion > 0.05:
#         return "Coins in Hand"

#     return "Colored and Textured Background"